In [1]:
# Import libraries
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import altair_saver 
import geopandas as gpd
from mpl_toolkits.basemap import Basemap
import folium
import warnings

# Filter out the specific warning you want to ignore
warnings.filterwarnings("ignore", message="iteritems is deprecated and will be removed in a future version")

C:\Users\pmarshal\Anaconda3\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# Regional Highlights

The vast majority of golf courses played are in North America and the United Kingdom (plus Ireland). This section will focus on the best courses in on these two regions.

## North America

Mike has played a grand total **717** different golf courses in Canada and the USA, with the vast majority located on the west coast and in Hawaii. This includes several gems like Cypress Point and Pebble Beach on the Monterey Peninsula in California, and Bandon and Pacific Dunes on Oregon Coast. 

```{figure} img/pacific-dunes.jpg
---
name: pacific_dunes
---
14th hole, Pacific Dunes Golf Course
```

In [2]:
# Set working directory
os.chdir('C://Users/pmarshal/Documents/golf_courses')
os.getcwd()

'C:\\Users\\pmarshal\\Documents\\golf_courses'

In [3]:
# Import data
course_master = pd.read_csv('data/master_golf_list.csv')


In [4]:
can_usa_df = course_master.loc[(course_master['country'] == 'Canada') | (course_master['country'] == 'United States of America')]
gdf_na_courses = gpd.GeoDataFrame(can_usa_df, geometry=gpd.points_from_xy(can_usa_df.longitude, can_usa_df.latitude))


In [5]:
top_rated = pd.read_csv('top_rated2.csv', index_col='Course Name')
#top_10 = top_10.reset_index()
top_rated

,City,Country,Personal Score,Best Score
Course Name,,,,
Cypress Point Golf Club,Monterey,USA,11.0,72
Pebble Beach Links,Pebble Beach,USA,10.0,75
Capilano Golf and Country Club,West Vancouver,Canada,10.0,65
Olympic Lake Course,San Francisco,USA,10.0,79
Pine Valley GC,Clementon,USA,10.0,75
Bandon Dunes GC,Bandon,USA,10.0,75
The National Golf Club,Woodbridge,Canada,10.0,79
Pacific Dunes GC,Bandon,USA,10.0,76
Torrey Pines -- South,San Diego,USA,9.7,73


Below is a look at some of the courses played that regularly rank in the top 100 in the US as per [Golf Digest](https://www.golfdigest.com/story/americas-100-greatest-golf-courses-ranking). The Canadian courses in Mike's top 10 all rank in the [top 5 courses in Canada](https://www.golfdigest.com/story/ranking-best-courses-in-canada). It's hard to say why the US courses are not ranked in the world's top-100 courses, when clearly the best US courses are worthy.

In [6]:
top_rated_courses = pd.read_csv('top_rated_courses.csv')


In [7]:
rank_cleaned2 = top_rated_courses.dropna(subset=['US Rank'])
# Creating the bar chart using Altair
us_rank = alt.Chart(rank_cleaned2).mark_bar(color='darkgreen', opacity=0.8).encode(
    alt.Y('Course Name:O', title=None, sort='x'),
    alt.X('US Rank:Q', title='Ranking', scale=alt.Scale(domain=[0,100])),
    tooltip=['Course Name:O', 'US Rank:Q'],  # adding tooltips to show the year and count
).properties(
    title='US Rank in 2023',
    width=300,
    height=200
)

# Combine the two charts
text3 = us_rank.mark_text(
    align='left',
    baseline='middle',
    size=9,
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('US Rank:Q'),  # Format as percentage
)
us_chart = us_rank + text3
us_chart

alt.LayerChart(...)

Below is a map showing the location of all the courses played in Canada and the US. You can zoom and scroll, and hover over points to see the names of the golf courses. 

In [8]:
m = folium.Map(location=[49.7, -117.0], zoom_start=5)

In [9]:
for course in gdf_na_courses["course_name"].unique():
    course_loc = gdf_na_courses[gdf_na_courses["course_name"]==course]
    folium.CircleMarker(
        location=[course_loc.latitude.values[0], course_loc.longitude.values[0]],
        tooltip = course_loc.course_name.values[0],
        radius = 4,
        color="blue",
        fill_color="red"
    ).add_to(m)
m

## The United Kingdom and Ireland

It's generally accepted that the modern game of golf originated in Scotland sometime in the middle ages. The first documented mention of the game in Scotland appears in a 1457 Act of the Scottish Parliament, issued by King James II, prohibiting the play of *gowf* as it was a distraction from archery practice for military purposes. Golf did gain popularity among royals and nobles in the coming decades; however, it was not until the late 1800's when the popularity of golf really boomed. 

The United Kingdom and Ireland are blessed with many of the best golf courses in the world. And many of these are classic links courses dotting the coast. Five of the top 10 best ranked golf courses in the world are in Ireland or the UK. It is truly a golf mecca. 

```{figure} img/st-andrews-old.jpg
---
name: st-andrews
---
St. Andrews -- Old Course
```

In [10]:
uk_df = course_master.loc[(course_master['country'] == 'United Kingdom') | (course_master['country'] == 'Ireland')]
gdf_uk_courses = gpd.GeoDataFrame(uk_df, geometry=gpd.points_from_xy(uk_df.longitude, uk_df.latitude))

In [11]:
# Import data
top_10 = pd.read_csv('top_10.csv', index_col='Course Name')
#top_10 = top_10.reset_index()
top_10

,City,Country,Personal Rating,Best Score,Par,Course Rating,Slope
Course Name,,,,,,,
Royal County Down GC,Newcastle,N. Ireland,11,74,71,75.0,142
Sunningdale Old,Sunningdale,England,10,80,70,72.7,139
Ballybunion Old,Ballybunion,Ireland,10,72,71,74.5,131
Turnberry -- Ailsa,Turnberry,Scotland,10,82,71,77.2,139
Muirfield,Gullane,Scotland,10,86,71,70.4,132
Royal Portrush GC -- Dunluce,Portrush,N. Ireland,10,75,72,76.0,140
The Old Course,St Andrews,Scotland,10,76,72,73.1,132
Royal Dornoch GC,Dornoch,Scotland,10,77,70,73.2,136
Carne GC,Belmullet,Ireland,10,81,72,72.6,124


Below is a map showing the location of all the courses played in Ireland and the UK. You can zoom and scroll, and hover over points to see the names of the golf courses. 

In [12]:
m2 = folium.Map(location=[53.6, -3.5], zoom_start=5.5)

In [13]:
for course in gdf_uk_courses["course_name"].unique():
    course_loc = gdf_uk_courses[gdf_uk_courses["course_name"]==course]
    folium.CircleMarker(
        location=[course_loc.latitude.values[0], course_loc.longitude.values[0]],
        tooltip = course_loc.course_name.values[0],
        radius = 4,
        color="blue",
        fill_color="red"
    ).add_to(m2)
m2